In [ ]:
import dlt
from pyspark.sql.functions import col, expr

In [ ]:
ambiente = 'dev'


## Bronze

In [ ]:
df_sales = spark.read.format("csv").option("header", "true").load("/FileStore/tmp/testes/sales_data.csv")
display(df_sales)

sale_id,sales_amount,sale_date,customer_id,product_id,quantity
1,100.50,2024-07-01,101,201,2
2,250.75,2024-07-02,102,202,1
3,300.00,2024-07-03,103,203,3
4,150.25,2024-07-04,104,204,1
5,200.00,2024-07-05,105,205,4
6,175.50,2024-07-06,106,206,2
7,225.75,2024-07-07,107,207,1
8,320.00,2024-07-08,108,208,3
9,180.25,2024-07-09,109,209,1
10,210.00,2024-07-10,110,210,4


In [ ]:
@dlt.table(
    name="bronze_sale",
    comment="Bronze sales",
    table_properties={"quality": "bronze"}
)
def bronze_sale():
  df_bronze = spark.read.format("csv").option("header", "true").load("/FileStore/tmp/testes/sales_data.csv")
  return df_bronze

: 

## Silver

In [ ]:
@dlt.table(
  name="silver_sale",
  comment="Silver sale",
  table_properties={"quality": "silver"}
)
def silver_sale():
  df_silver = dlt.read_stream("bronze_teste")
  df_cleaned = df_silver.filter(col("sales_amount").isNotNull()) \
                    .withColumn("sales_amount", col("sales_amount").cast("float")) \
                    .withColumn("sale_date", expr("to_date(sale_date, 'yyyy-MM-dd')"))
  return df_cleaned

## Gold

In [ ]:
@dlt.table(
  name="gold_sale",
  comment="Gold sale",
  table_properties={"quality": "gold"}
)
def gold_sale():
  df_gold = dlt.read_stream("silver_teste")
  df_summary = df_gold.groupBy("sale_date") \
                   .agg(
                       expr("sum(sales_amount)").alias("total_sales"),
                       expr("count(*)").alias("total_transactions")
                   )
  return df_summary